In [13]:
#@title Day 1: Trebuchet?!
from google.colab import files
with open('23-1.txt','r') as f:
  ls = f.read().splitlines()

print(sum(map(lambda d: int(d[0]+d[-1]),[[c for c in l if '/' < c < ':'] for l in ls])))

ns = {n: n[0]+str(i)+n[-1] for (i,n) in enumerate('zero,one,two,three,four,five,six,seven,eight,nine'.split(','))}
[ls:= [l.replace(k,ns[k]) for l in ls] for k in ns]
print(sum([int(re.search('\d',l).group()+re.search('\d',l[::-1]).group()) for l in ls]))


54573
54591


In [ ]:
#@title Day 2: Cube Conundrum
import re, math

maxes = {'red': 12, 'green': 13, 'blue': 14}

from google.colab import files
with open('23-2.txt','r') as f:
  gs = [[{p.strip().split(' ')[1]: int(p.strip().split(' ')[0])
          for p in r.split(',')}
          for r in l.split(':')[1].split(';')]
          for l in f.read().splitlines()]

valid = [i+1 for (i,g) in enumerate(gs)
             if all([all([maxes[k]>=r[k] for k in r]) for r in g])]
print(sum(valid))

powers = [math.prod([max([r.get(k,0) for r in g]) for k in maxes]) for g in gs]
print(sum(powers))

2727
56580
